In [7]:
!pip install opencv-python mediapipe numpy

In [21]:
import cv2
import mediapipe as mp
import numpy as np
import time


In [22]:
mp_hands=mp.solutions.hands 
mp_drawing=mp.solutions.drawing_utils
hands= mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

#red in BGR
draw_color= (0,0,255)
brush_thickness =8
eraser_thickness=40

#screen top having colour choices
colors = {
    'Red': (0,0,255),
    'Green': (0,255,0),
    'Blue': (255,0,0),
    'Eraser': (0,0,0)
}


In [23]:
cap=cv2.VideoCapture(0)
canvas=None
prev_x,prev_y=0,0
last_seen= time.time()  #auto stop

In [24]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame=cv2.flip(frame, 1)
    h,w,_=frame.shape
    if canvas is None:
        canvas= np.zeros_like(frame)

    rgb=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result =hands.process(rgb)

    #draw color buttons at top
    cv2.rectangle(frame, (0,0),(w,70),(50, 50,50),-1)
    x_pos = 10
    for color_name, color_val in colors.items():
        cv2.rectangle(frame, (x_pos, 10), (x_pos + 100, 60), color_val, -1)
        cv2.putText(frame, color_name, (x_pos + 10, 45),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        x_pos += 120

    if result.multi_hand_landmarks:
        last_seen = time.time()  #time is reset when a hand is detected

        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            #get index & middle finger postn
            x1 = int(hand_landmarks.landmark[8].x * w)
            y1 = int(hand_landmarks.landmark[8].y * h)
            x2 = int(hand_landmarks.landmark[12].x * w)
            y2 = int(hand_landmarks.landmark[12].y * h)
            distance = np.hypot(x2 - x1, y2 - y1)

            #check if finger is selecting color from top
            if y1 < 70:
                if 10 < x1 < 110:
                    draw_color = (0, 0, 255)#red
                elif 130 < x1 < 230:
                    draw_color = (0, 255, 0) #green
                elif 250 < x1 < 350:
                    draw_color = (255, 0, 0) #blue
                elif 370 < x1 < 470:
                    draw_color = (0, 0, 0)  #eraser
                prev_x, prev_y = 0, 0  #tracking prev locations for smooth lines

            #draw
            elif distance > 40:
                cv2.circle(frame, (x1, y1), 8, draw_color, -1)
                if prev_x == 0 and prev_y == 0:
                    prev_x, prev_y = x1, y1
                thickness = eraser_thickness if draw_color == (0, 0, 0) else brush_thickness
                cv2.line(canvas, (prev_x, prev_y), (x1, y1), draw_color, thickness)
                prev_x, prev_y = x1, y1
            else:
                prev_x, prev_y = 0, 0

    #autostop timer
    elif time.time()-last_seen>20:
        print("No hand detected for 20 seconds...stopping automatically.")
        break

    #combine canvas &frame
    gray = cv2.cvtColor(canvas,cv2.COLOR_BGR2GRAY)
    _, inv = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY_INV)#threshold step
    inv = cv2.cvtColor(inv, cv2.COLOR_GRAY2BGR)
    frame = cv2.bitwise_and(frame, inv)
    frame = cv2.bitwise_or(frame, canvas)

    cv2.imshow("AI Hand Painter", frame)
    cv2.imshow("Canvas", canvas)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Quit command received.. exiting.")
        break

cap.release()
cv2.destroyAllWindows()


No hand detected for 20 seconds...stopping automatically.
